<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.



**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
!pip install pytorch-pretrained-bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 123 kB 8.2 MB/s 
     |████████████████████████████████| 132 kB 10.5 MB/s 
     |████████████████████████████████| 9.1 MB 55.1 MB/s 
     |████████████████████████████████| 79 kB 4.4 MB/s 
     |████████████████████████████████| 140 kB 46.0 MB/s 
     |████████████████████████████████| 127 kB 45.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 8.7 MB/s 
     |████████████████████████████████| 120 kB 64.7 MB/s 
     |████████████████████████████████| 6.6 MB 51.6 MB/s 


In [3]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.4 MB/s 


## Подготовка

In [4]:
import pandas as pd #устанавливаем Pandas
import numpy as np #устанавливаем Numpy
import torch #импортируем torch
import transformers #импортируем transformesr
from tqdm import notebook #для отслеживания прогресса
from sklearn.linear_model import LogisticRegression #логистическая регрессия 
from sklearn.model_selection import cross_val_score #кросс-валидация
from sklearn.model_selection import train_test_split # для разделения датасета на тренирочную и тестовую
from sklearn.metrics import f1_score #для нахождения точности f1
from catboost import CatBoostClassifier #для catboost
import re #для работы с регулярными выражениями
import spacy #обработка естественного языка
from sklearn.feature_extraction.text import TfidfVectorizer #для оценки важности

In [5]:
data = pd.read_csv("toxic_comments.csv").sample(5000).reset_index(drop = True ) #скачаем датасет и режем его, иначе оперативки не хватает
data.sample(5) #выведем 5 случайных строк

,Unnamed: 0,text,toxic
3682,93381,"""\nIt happens to the best of us. ) λόγοςπράξι...",0
3255,147834,Moviemaker92 11:50 EST 3/23/07,0
2671,89860,FYI: Please see this: User talk:Central. If th...,0
4282,8171,Marijuna and the Brain\nWe need some help over...,0
268,15038,Brianna Marie Motz is AMAZING and Funny:),0


In [6]:
data.info() #информация о датасете

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5000 non-null   int64 
 1   text        5000 non-null   object
 2   toxic       5000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 117.3+ KB


In [7]:
data.isna().sum() #количество пропусков

Unnamed: 0    0
text          0
toxic         0
dtype: int64

In [8]:
data.duplicated().sum() #количество дубликатов

0

Создаем токенизатор для BERT, лемматизация не требуется в этом случае

In [9]:
#выгружаем нужные параметры
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel,
                                                    transformers.DistilBertTokenizer, 'distilbert-base-uncased')

In [10]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights) #иницализируем токенизатор из класса tokenizer_class

In [11]:
#преобразуем текст в номера токенов из словаря
tokenized = data['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True)) #максимальная длинна входа 512

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values]) #условие BERT

attention_mask = np.where(padded != 0, 1, 0) #поясним моделе, что 0 не несут значимой информации

Инициализируем саму модель 

In [12]:
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
batch_size = 50 #номер батчей
embeddings = [] #контейнер эмбендингов
for i in notebook.tqdm(range(padded.shape[0] // batch_size)): 
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/100 [00:00<?, ?it/s]

In [14]:
features = np.concatenate(embeddings) #признаки

## Обучение

In [15]:
features_train, features_test, target_train, target_test = train_test_split(
    features, data['toxic'], test_size=0.25, random_state=12345) #создаем тренировочные и тестовые данные

In [16]:
model = LogisticRegression(random_state=12345, solver="liblinear")
model.fit(features_train, target_train) #обучение на тренировочных
predict_ = model.predict(features_test) #предсказываем на тестовых
print("F1 метрика для  Логистической регрессии - ", f1_score(target_test, predict_))

F1 метрика для  Логистической регрессии -  0.6210526315789474


In [17]:
cat = CatBoostClassifier(random_state=12345, task_type='GPU')
cat.fit(features_train, target_train, verbose=100) #обучение на тренировочных
predict_ = cat.predict(features_test) #предсказываем на тестовых
print("F1 метрика для  СatBoost - ", f1_score(target_test, predict_))

Learning rate set to 0.031347
0:	learn: 0.6521454	total: 129ms	remaining: 2m 8s
100:	learn: 0.1286729	total: 6.16s	remaining: 54.8s
200:	learn: 0.0912432	total: 12s	remaining: 47.7s
300:	learn: 0.0674511	total: 17.8s	remaining: 41.3s
400:	learn: 0.0508413	total: 23.6s	remaining: 35.3s
500:	learn: 0.0379836	total: 29.6s	remaining: 29.4s
600:	learn: 0.0284841	total: 35.6s	remaining: 23.6s
700:	learn: 0.0224994	total: 41.6s	remaining: 17.7s
800:	learn: 0.0180416	total: 47.6s	remaining: 11.8s
900:	learn: 0.0147794	total: 53.6s	remaining: 5.89s
999:	learn: 0.0123278	total: 59.5s	remaining: 0us
F1 метрика для  СatBoost -  0.5433526011560693


Метрика слабовата, при повышении размера датасета максимальное значение было 0,7. В рамках работы тяжело обучить модель на всех данных, а она думаю давала бы достаточно высокий результат. Попробую просто провести обработку текста по теории в модуле и посчитать значение.

In [18]:
nlp = spacy.load('en_core_web_sm')  #английский лемматизатор
def clear_text(text): #функция для очистки текста
    return " ".join(re.sub(r'[^a-zA-Z]', ' ', text.lower()).split()) #объединяем разрешенные элементу в строку

def lemmatize(text): #лемматищация
    global nlp #загружаем функцию
    doc = nlp(text) #лемматизируем текст
    return " ".join([token.lemma_ for token in doc]) #возвращаем список лемм

data['clear_text'] = data['text'].apply(clear_text) #созздаем очищенный столбец 
data['lemmatized_text'] = data['clear_text'].apply(lemmatize) #отсавляем лемммы

features = data['lemmatized_text'] #признак (текст)
target = data['toxic'] #целевой признак

In [19]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=12345, stratify=target) #снова делим

In [20]:
cat.fit(pd.DataFrame(features_train), target_train, text_features=['lemmatized_text'], verbose=100) #обучаем
predict_ = cat.predict(pd.DataFrame(features_test)) #предсказываем на тестовых #предсказываем
print("F1 метрика для  СatBoost - ", f1_score(target_test, predict_)) #выводим f1

Learning rate set to 0.031347
0:	learn: 0.6503171	total: 53.1ms	remaining: 53.1s
100:	learn: 0.1635683	total: 3s	remaining: 26.7s
200:	learn: 0.1382996	total: 5.78s	remaining: 23s
300:	learn: 0.1220467	total: 8.58s	remaining: 19.9s
400:	learn: 0.1099098	total: 11.4s	remaining: 17s
500:	learn: 0.1006882	total: 14.2s	remaining: 14.2s
600:	learn: 0.0934699	total: 17s	remaining: 11.3s
700:	learn: 0.0881156	total: 19.9s	remaining: 8.47s
800:	learn: 0.0837444	total: 22.7s	remaining: 5.63s
900:	learn: 0.0806013	total: 25.5s	remaining: 2.8s
999:	learn: 0.0780037	total: 28.3s	remaining: 0us
F1 метрика для  СatBoost -  0.6534653465346535


Странно, но очисткой текста и лемматизацией, мы смогли получить нужное значение матрики f1.

## Выводы

Таким образом, в рамках нашей работы, мы смогли получить требуемое значение. Задание было сделано двумя способами, с использованием BERT и без него.